In [1]:
import os
from PIL import Image
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from torchvision import models, transforms

In [2]:
# Load GoogleNet model pretrained on ImageNet, set it to evaluation mode, and use the CPU
gn = models.googlenet(pretrained=True)
gn.eval()  # set the model to evaluation mode

# Define the transformation to convert images to tensor
img_transform = transforms.ToTensor()

# Create a DataFrame to store the output features and target labels
df = pd.DataFrame(columns=list(range(1000)))  # assuming 1000 output features
target = pd.DataFrame(columns=['target'])

d:\Major1\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Major1\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
for i in os.listdir('final_image_data'):
    for j in os.listdir('final_image_data/' + i):
        for k in os.listdir('final_image_data/' + i + '/' + j):
            # Load and transform the image
            im = Image.open('final_image_data/' + i + '/' + j + '/' + k)
            transformed_img = img_transform(im).unsqueeze(0)  # Add batch dimension
            
            # Forward pass on the model using CPU
            features = gn.forward(torch.autograd.Variable(transformed_img)).detach().numpy()
            
            # Convert the feature and target row into DataFrames
            feature_df = pd.DataFrame(features, columns=list(range(1000)))
            target_df = pd.DataFrame([int(j)], columns=['target'])
            
            # Concatenate feature_df and target_df to df and target, respectively
            df = pd.concat([df, feature_df], ignore_index=True)
            target = pd.concat([target, target_df], ignore_index=True)

C:\Users\RIYA SUMRA\AppData\Local\Temp\ipykernel_7788\4164665976.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, feature_df], ignore_index=True)


In [4]:
# x_train,x_test,y_train,y_test=train_test_split(df,target[0],train_size=0.8)
#test-train split
x_train, x_test, y_train, y_test = train_test_split(df, target['target'], train_size=0.8, random_state=42)

In [5]:
model=svm.SVC(kernel='linear',probability=True)
#SVM model

In [6]:
# Ensure y_train and y_test are integer arrays
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Fit the model and evaluate
model.fit(x_train, y_train)
print("Model Accuracy on test set:", model.score(x_test, y_test))
print("Model Accuracy on train set:", model.score(x_train, y_train))


Model Accuracy on test set: 0.945
Model Accuracy on train set: 0.9689583333333334


In [ ]:
import joblib
from sklearn import svm
import joblib
joblib.dump(model, 'my_svm.pkl')  # Save the model using joblib

['my_svm.pkl']